In [1]:
from common import *

DAY = 10

In [2]:
show_task(DAY)

<article class="day-desc"><h2>--- Day 10: Pipe Maze ---</h2><p>You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.</p>
<p>You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "<a href="https://en.wikipedia.org/wiki/Hot_spring" target="_blank">Hot Springs</a>" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.</p>
<p>The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.</p>
<p>Scanning the area, you discover that the entire field you're standing on is <span title="Manufactured by Hamilton and Hilbert Pipe Company">densely packed with pipes</span>; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).</p>
<p>The pipes are arranged in a two-dimensional grid of <em>tiles</em>:</p>
<ul>
<li><code>|</code> is a <em>vertical pipe</em> connecting north and south.</li>
<li><code>-</code> is a <em>horizontal pipe</em> connecting east and west.</li>
<li><code>L</code> is a <em>90-degree bend</em> connecting north and east.</li>
<li><code>J</code> is a <em>90-degree bend</em> connecting north and west.</li>
<li><code>7</code> is a <em>90-degree bend</em> connecting south and west.</li>
<li><code>F</code> is a <em>90-degree bend</em> connecting south and east.</li>
<li><code>.</code> is <em>ground</em>; there is no pipe in this tile.</li>
<li><code>S</code> is the <em>starting position</em> of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.</li>
</ul>
<p>Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is <em>one large, continuous loop</em>.</p>
<p>For example, here is a square loop of pipe:</p>
<pre><code>.....
.F-7.
.|.|.
.L-J.
.....
</code></pre>
<p>If the animal had entered this loop in the northwest corner, the sketch would instead look like this:</p>
<pre><code>.....
.<em>S</em>-7.
.|.|.
.L-J.
.....
</code></pre>
<p>In the above diagram, the <code>S</code> tile is still a 90-degree <code>F</code> bend: you can tell because of how the adjacent pipes connect to it.</p>
<p>Unfortunately, there are also many pipes that <em>aren't connected to the loop</em>! This sketch shows the same loop as above:</p>
<pre><code>-L|F7
7S-7|
L|7||
-L-J|
L|-JF
</code></pre>
<p>In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to <code>S</code>, pipes those pipes connect to, pipes <em>those</em> pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including <code>S</code>, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).</p>
<p>Here is a sketch that contains a slightly more complex main loop:</p>
<pre><code>..F7.
.FJ|.
SJ.L7
|F--J
LJ...
</code></pre>
<p>Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:</p>
<pre><code>7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
</code></pre>
<p>If you want to <em>get out ahead of the animal</em>, you should find the tile in the loop that is <em>farthest</em> from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps <em>along the loop</em> to reach from the starting point - regardless of which way around the loop the animal went.</p>
<p>In the first example with the square loop:</p>
<pre><code>.....
.S-7.
.|.|.
.L-J.
.....
</code></pre>
<p>You can count the distance each tile in the loop is from the starting point like this:</p>
<pre><code>.....
.012.
.1.3.
.23<em>4</em>.
.....
</code></pre>
<p>In this example, the farthest point from the start is <code><em>4</em></code> steps away.</p>
<p>Here's the more complex loop again:</p>
<pre><code>..F7.
.FJ|.
SJ.L7
|F--J
LJ...
</code></pre>
<p>Here are the distances for each tile on that loop:</p>
<pre><code>..45.
.236.
01.7<em>8</em>
14567
23...
</code></pre>
<p>Find the single giant loop starting at <code>S</code>. <em>How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?</em></p>
</article>

In [6]:
lines = '''..F7.
.FJ|.
SJ.L7
|F--J
LJ...'''.splitlines()
lines

['..F7.', '.FJ|.', 'SJ.L7', '|F--J', 'LJ...']

In [36]:
lines = get_input_lines(DAY)

In [37]:
paths = {
    'U': '|LJS',
    'D': '|7FS',
    'L': '-J7S',
    'R': '-LFS',
}
inverted_paths = {
    'U': 'D',
    'D': 'U',
    'L': 'R',
    'R': 'L',
}

In [38]:
dmap = [[-1] * len(line) for line in lines]

for y, line in enumerate(lines):
    for x, c in enumerate(line):
        if c == 'S':
            dmap[y][x] = 0

# dmap

In [39]:
def dmap_step(dmap: list[list[int]]) -> bool:
    changed = False
    for y, dline in enumerate(dmap):
        for x, d in enumerate(dline):
            if d == -1:
                continue
            c = lines[y][x]
            for ydiff, xdiff, direction in [(-1, 0, 'U'), (1, 0, 'D'), (0, -1, 'L'), (0, 1, 'R')]:
                new_y, new_x = y + ydiff, x + xdiff

                # do not go through walls
                if not (0 <= new_y < len(dmap)) or not (0 <= new_x < len(dline)):
                    continue
                new_c = lines[new_y][new_x]

                # connections check
                if c not in paths[direction] or new_c not in paths[inverted_paths[direction]]:
                    continue

                new_d = d + 1
                if dmap[new_y][new_x] == -1 or dmap[new_y][new_x] > new_d:
                    dmap[new_y][new_x] = new_d
                    changed = True
    return changed

def show_dmap(dmap: list[list[int]]):
    for line in dmap:
        for num in line:
            num = num if num != -1 else '.'
            print(f'{num:>2} ', end='')
        print()


In [41]:
while dmap_step(dmap):
    pass

# show_dmap(dmap)

In [42]:
result = max(max(line) for line in dmap)
result

6867

In [43]:
send_result(DAY, 1, result)

"That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two] (solved in 78:30)"

In [44]:
show_task(DAY, 2)

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is <em>within the area enclosed by the loop</em>?</p>
<p>To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:</p>
<pre><code>...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
</code></pre>
<p>The above loop encloses merely <em>four tiles</em> - the two pairs of <code>.</code> in the southwest and southeast (marked <code>I</code> below). The middle <code>.</code> tiles (marked <code>O</code> below) are <em>not</em> in the loop. Here is the same loop again with those regions marked:</p>
<pre><code>...........
.S-------7.
.|F-----7|.
.||<em>OOOOO</em>||.
.||<em>OOOOO</em>||.
.|L-7<em>O</em>F-J|.
.|<em>II</em>|<em>O</em>|<em>II</em>|.
.L--J<em>O</em>L--J.
.....<em>O</em>.....
</code></pre>
<p>In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, <code>I</code> is still within the loop and <code>O</code> is still outside the loop:</p>
<pre><code>..........
.S------7.
.|F----7|.
.||<em>OOOO</em>||.
.||<em>OOOO</em>||.
.|L-7F-J|.
.|<em>II</em>||<em>II</em>|.
.L--JL--J.
..........
</code></pre>
<p>In both of the above examples, <code><em>4</em></code> tiles are enclosed by the loop.</p>
<p>Here's a larger example:</p>
<pre><code>.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
</code></pre>
<p>The above sketch has many random bits of ground, some of which are in the loop (<code>I</code>) and some of which are outside it (<code>O</code>):</p>
<pre><code><em>O</em>F----7F7F7F7F-7<em>OOOO</em>
<em>O</em>|F--7||||||||FJ<em>OOOO</em>
<em>O</em>||<em>O</em>FJ||||||||L7<em>OOOO</em>
FJL7L7LJLJ||LJ<em>I</em>L-7<em>OO</em>
L--J<em>O</em>L7<em>III</em>LJS7F-7L7<em>O</em>
<em>OOOO</em>F-J<em>II</em>F7FJ|L7L7L7
<em>OOOO</em>L7<em>I</em>F7||L7|<em>I</em>L7L7|
<em>OOOOO</em>|FJLJ|FJ|F7|<em>O</em>LJ
<em>OOOO</em>FJL-7<em>O</em>||<em>O</em>||||<em>OOO</em>
<em>OOOO</em>L---J<em>O</em>LJ<em>O</em>LJLJ<em>OOO</em>
</code></pre>
<p>In this larger example, <code><em>8</em></code> tiles are enclosed by the loop.</p>
<p>Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:</p>
<pre><code>FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
</code></pre>
<p>Here are just the tiles that are <em>enclosed by the loop</em> marked with <code>I</code>:</p>
<pre><code>FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ<em>I</em>F7FJ-
L---JF-JLJ<em>IIII</em>FJLJJ7
|F|F-JF---7<em>III</em>L7L|7|
|FFJF7L7F-JF7<em>II</em>L---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
</code></pre>
<p>In this last example, <code><em>10</em></code> tiles are enclosed by the loop.</p>
<p>Figure out whether you have time to search for the nest by calculating the area within the loop. <em>How many tiles are enclosed by the loop?</em></p>
</article>

In [73]:
# lines = '''.F----7F7F7F7F-7....
# .|F--7||||||||FJ....
# .||.FJ||||||||L7....
# FJL7L7LJLJ||LJ.L-7..
# L--J.L7...LJS7F-7L7.
# ....F-J..F7FJ|L7L7L7
# ....L7.F7||L7|.L7L7|
# .....|FJLJ|FJ|F7|.LJ
# ....FJL-7.||.||||...
# ....L---J.LJ.LJLJ...'''.splitlines()

lines = get_input_lines(DAY)

dmap = [[-1] * len(line) for line in lines]

for y, line in enumerate(lines):
    for x, c in enumerate(line):
        if c == 'S':
            dmap[y][x] = 0

while dmap_step(dmap):
    pass

# show_dmap(dmap)

In [74]:
def create_map(y, x, value: int = -1) -> list[list[int]]:
    line = [value] * x
    return [line[:] for _ in range(y)]

pmap = create_map(2 + len(lines) * 2, 2 + len(lines[0]) * 2)

def find_value(dmap: list[list[int]], value=0) -> tuple[int, int]:
    for y, line in enumerate(dmap):
        for x, d in enumerate(line):
            if d == value:
                return y, x
    raise ValueError('Not found')


def nearest(dmap: list[list[int]], y: int, x: int, diff: int = -1) -> list[tuple[int, int]]:
    result = []
    num = dmap[y][x]
    for ydiff, xdiff in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_y, new_x = y + ydiff, x + xdiff

        # do not go through walls
        if not (0 <= new_y < len(dmap)) or not (0 <= new_x < len(dmap[0])):
            continue

        new_num = dmap[new_y][new_x]
        if new_num != -1 and new_num == num + diff:
            result.append((new_y, new_x))
    return result

max_d = max(max(line) for line in dmap)
start_y, start_x = find_value(dmap, max_d)
pmap[2 + start_y * 2][2 + start_x * 2] = dmap[start_y][start_x]

for y, x in nearest(dmap, start_y, start_x):
    pmap[2 + start_y * 2 + (y - start_y)][2 + start_x * 2 + (x - start_x)] = dmap[y][x]
    pmap[2 + y * 2][2 + x * 2] = dmap[y][x]
    while True:
        num = dmap[y][x]
        for new_y, new_x in nearest(dmap, y, x):
            new_num = dmap[new_y][new_x]
            pmap[2 + new_y * 2][2 + new_x * 2] = new_num
            pmap[2 + y * 2 + (new_y - y)][2 + x * 2 + (new_x - x)] = new_num
            y, x = new_y, new_x
            break
        else:
            break

# show_dmap(pmap)

In [75]:
pmap[0][0] = 'O'

def all_nearest(dmap: list[list[int]], y: int, x: int) -> list[tuple[int, int, int]]:
    result = []
    for ydiff, xdiff in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_y, new_x = y + ydiff, x + xdiff

        # do not go through walls
        if not (0 <= new_y < len(dmap)) or not (0 <= new_x < len(dmap[0])):
            continue

        new_num = dmap[new_y][new_x]
        result.append((new_y, new_x, new_num))
    return result


def pmap_step(pmap: list[list[int]]) -> bool:
    changed = False
    for y, line in enumerate(pmap):
        for x, value in enumerate(line):
            if value != 'O':
                continue
            for new_y, new_x, new_value in all_nearest(pmap, y, x):
                if new_value != -1:
                    continue
                pmap[new_y][new_x] = 'O'
                changed = True
    return changed

while pmap_step(pmap):
    pass

result = 0
for y, line in enumerate(pmap):
    for x, value in enumerate(line):
        if value == -1 and x % 2 == 0 and y % 2 == 0:
            result += 1
print(result)
# show_dmap(pmap)

595


In [76]:
send_result(DAY, 2, result)

"That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 10! (solved in 96:47)"